In [1]:
%%bash
module list

Currently Loaded Modulefiles:
 1) texlive/2019   2) anaconda/3_5.3.1   3) gromacs2020/2020.0(default)  


In [2]:
%matplotlib inline

In [3]:
import os
import asyncio
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda

In [4]:
import aimmd
import aimmd.distributed as aimmdd

# GMX engine

In [5]:
n_engines = 4

In [6]:
scratch_dir = "/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/"
wdirs = [os.path.join(scratch_dir, f"engine_wdir{i}") for i in range(n_engines)]

In [7]:
for d in wdirs:
    if not os.path.isdir(d):
        os.mkdir(d)

## `GmxEngine` is a thin wrapper around gromacs
- it has a `prepare()` method (which will call `grompp`) and multiple methods to then rujn the simulation, namely `run()`, `run_walltime()` and `run_nsteps()`

In [8]:
engines = [aimmdd.GmxEngine(gro_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                            top_file=os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                            ndx_file=None,
                            #mdrun_extra_args="-nt 2",  # use this if your version of GMX is compiled with MPI support
                            mdrun_extra_args="-ntomp 2",  # use this for GMX without MPI support
                            )
           for _ in range(n_engines)]

In [9]:
mdps = [aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp"))
        for _ in range(n_engines)]

In [10]:
mdps[0].changed

False

In [11]:
mdps[0]

{'title': ['test'], 'cpp': ['/lib/cpp'], 'include': ['-I../top'], 'define': [], 'integrator': ['md-vv'], 'dt': 0.002, 'nsteps': -1, 'nstxout': 10, 'nstvout': 10, 'nstlog': 10, 'nstenergy': 10, 'nstxout-compressed': 10, 'compressed-x-grps': ['Protein'], 'energygrps': ['Protein', 'SOL'], 'nstlist': 10, 'ns-type': ['grid'], 'cutoff-scheme': ['Verlet'], 'rlist': 1.1, 'coulombtype': ['PME'], 'rcoulomb': 1.1, 'rvdw': 1.1, 'tcoupl': ['Berendsen'], 'tc-grps': ['Protein', 'SOL'], 'tau-t': [0.1, 0.1], 'ref-t': [300.0, 300.0], 'Pcoupl': ['Berendsen'], 'tau-p': 1.0, 'compressibility': [4.5e-05], 'ref-p': [1.0], 'gen-vel': ['no'], 'gen-temp': 300.0, 'gen-seed': 173529, 'constraints': ['all-bonds']}

In [12]:
#for mdp in mdps:
#    mdp['nstvout'] = 10
#    mdp["nstxout"] = 10

In [13]:
mdps[0]

{'title': ['test'], 'cpp': ['/lib/cpp'], 'include': ['-I../top'], 'define': [], 'integrator': ['md-vv'], 'dt': 0.002, 'nsteps': -1, 'nstxout': 10, 'nstvout': 10, 'nstlog': 10, 'nstenergy': 10, 'nstxout-compressed': 10, 'compressed-x-grps': ['Protein'], 'energygrps': ['Protein', 'SOL'], 'nstlist': 10, 'ns-type': ['grid'], 'cutoff-scheme': ['Verlet'], 'rlist': 1.1, 'coulombtype': ['PME'], 'rcoulomb': 1.1, 'rvdw': 1.1, 'tcoupl': ['Berendsen'], 'tc-grps': ['Protein', 'SOL'], 'tau-t': [0.1, 0.1], 'ref-t': [300.0, 300.0], 'Pcoupl': ['Berendsen'], 'tau-p': 1.0, 'compressibility': [4.5e-05], 'ref-p': [1.0], 'gen-vel': ['no'], 'gen-temp': 300.0, 'gen-seed': 173529, 'constraints': ['all-bonds']}

In [14]:
mdps[0].changed

False

In [15]:
import time

In [16]:
start = time.time()
await asyncio.gather(*(e.prepare(starting_configuration=None, workdir=wdir, deffnm="blib", run_config=mdp)
                               for e, wdir, mdp in zip(engines, wdirs, mdps)
                               )
                            )
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.21573829650878906 s


In [17]:
walltime = 0.02 # 0.01 h = 36 s
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(walltime) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


time elapsed: 75.4235634803772 s


In [22]:
from state_funcs import descriptor_func

descriptor_func_wrap = aimmdd.TrajectoryFunctionWrapper(descriptor_func, {"scratch_dir": "/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/"})

In [23]:
cv = await descriptor_func_wrap(trajs[0])

In [24]:
cv

array([[-0.1878258 , -1.3769007 ],
       [-0.1701046 , -1.4815865 ],
       [-0.18295437, -1.5012401 ],
       ...,
       [ 2.655287  , -1.552482  ],
       [ 2.7547574 , -1.5252008 ],
       [ 2.8538456 , -1.5856383 ]], dtype=float32)

In [25]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9153 frames (== 91530 integration steps)
engine 1 did 9233 frames (== 92330 integration steps)
engine 2 did 9169 frames (== 91690 integration steps)
engine 3 did 9121 frames (== 91210 integration steps)


In [26]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(10) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.8636863231658936 s


/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [27]:
cv = await descriptor_func_wrap(trajs[0])

In [28]:
cv

array([[ 2.8538456, -1.5856383],
       [ 2.8715212, -1.5738745]], dtype=float32)

In [29]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9155 frames (== 91550 integration steps)
engine 1 did 9235 frames (== 92350 integration steps)
engine 2 did 9171 frames (== 91710 integration steps)
engine 3 did 9123 frames (== 91230 integration steps)


In [24]:
start = time.time()
trajs = await asyncio.gather(*(e.run_steps(10) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.8220570087432861 s


/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


In [31]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[ 2.8715212 -1.5738745]
 [ 2.7258203 -1.4065523]]


In [32]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9157 frames (== 91570 integration steps)
engine 1 did 9237 frames (== 92370 integration steps)
engine 2 did 9173 frames (== 91730 integration steps)
engine 3 did 9125 frames (== 91250 integration steps)


In [33]:
start = time.time()
trajs = await asyncio.gather(*(e.run_walltime(0.001) for e in engines))
end = time.time()
print(f"time elapsed: {end-start} s")

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/MDAnalysis/topology/tpr/utils.py:389: DeprecationWarning: TPR files index residues from 0. From MDAnalysis version 2.0, resids will start at 1 instead. If you wish to keep indexing resids from 0, please set `tpr_resid_from_one=False` as a keyword argument when you create a new Topology or Universe.
  warnings.warn("TPR files index residues from 0. "


time elapsed: 4.6737377643585205 s


In [34]:
cv = await descriptor_func_wrap(trajs[0])
print(cv)

[[ 2.7258203  -1.4065523 ]
 [ 2.9640176  -1.6019828 ]
 [ 3.1300347  -1.6560661 ]
 [ 3.0382085  -1.5191604 ]
 [ 2.9377978  -1.6072451 ]
 [ 3.0410552  -1.5922588 ]
 [ 3.0146089  -1.6349123 ]
 [ 2.8225913  -1.6294135 ]
 [ 2.736512   -1.6612377 ]
 [ 2.8564024  -1.6286631 ]
 [ 2.8337288  -1.7651278 ]
 [ 2.7057264  -1.383769  ]
 [ 2.6955357  -1.4326061 ]
 [ 2.7578928  -1.3578266 ]
 [ 2.6465063  -1.0991731 ]
 [ 2.5426295  -1.0041287 ]
 [ 2.605516   -1.0120215 ]
 [ 2.660292   -0.942427  ]
 [ 2.5447655  -0.92581433]
 [ 2.5263658  -0.93455046]
 [ 2.517192   -1.0552974 ]
 [ 2.6707606  -1.2326305 ]
 [ 2.4980965  -1.1868676 ]
 [ 2.553026   -1.2357893 ]
 [ 2.5744507  -1.3263937 ]
 [ 2.532144   -1.4108065 ]
 [ 2.4662619  -1.3335556 ]
 [ 2.5194192  -1.3929738 ]
 [ 2.4789493  -1.4576728 ]
 [ 2.3886921  -1.3002166 ]
 [ 2.3218286  -1.6068863 ]
 [ 2.2413588  -1.4320781 ]
 [ 2.2599635  -1.4697754 ]
 [ 1.9392961  -1.5107043 ]
 [ 2.0164266  -1.4405842 ]
 [ 1.8281262  -1.4414577 ]
 [ 1.7932687  -1.2970252 ]
 

In [35]:
for i, e in enumerate(engines):
    print(f"engine {i} did {e.frames_done} frames (== {e.steps_done} integration steps)")

engine 0 did 9628 frames (== 96280 integration steps)
engine 1 did 9716 frames (== 97160 integration steps)
engine 2 did 9644 frames (== 96440 integration steps)
engine 3 did 9596 frames (== 95960 integration steps)


# propagator until state

In [13]:
import mdtraj as mdt
import numpy as np
"""
def psi(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_psi(traj)
    return psi

def alpha_R(traj):
    traj = mdt.load(traj.trajectory_file, 
                    top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),  # mdt can not work with tprs, so we use theinitial gro for now
                    )
    psi = mdt.compute_dihedrals(traj, indices=[[6,8,14,16]])[:, 0]
    phi = mdt.compute_dihedrals(traj, indices=[[4,6,8,14]])[:, 0]
    state = np.full_like(psi, False, dtype=bool)
    # phi: -pi -> 0 
    # psi: > -50 but smaller 30 degree
    deg = 180/np.pi
    state[(phi <= 0) & (-50/deg <= psi) & (psi <= 30/deg)] = True
    return state

wrapped_alphaR = arcdd.trajectory.TrajectoryFunctionWrapper(alpha_R)
wrapped_psi = arcdd.trajectory.TrajectoryFunctionWrapper(psi)
"""

from state_funcs import alpha_R, C7_eq
wrapped_alphaR = aimmdd.TrajectoryFunctionWrapper(alpha_R, call_kwargs={"scratch_dir": scratch_dir})
wrapped_C7_eq = aimmdd.TrajectoryFunctionWrapper(C7_eq, call_kwargs={"scratch_dir": scratch_dir})

In [14]:
import aimmd.distributed.logic

In [15]:
propagators = [aimmd.distributed.logic.PropagatorUntilAnyState(states=[wrapped_alphaR, wrapped_C7_eq],
                                                               engine_cls=aimmdd.GmxEngine,
                                                               engine_kwargs={"gro_file":os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                                                              "top_file":os.path.join(scratch_dir, "gmx_infiles/topol.top"),
                                                                               "mdrun_extra_args": "-nt 4",
                                                                              },
                                                               run_config=aimmdd.MDP(os.path.join(scratch_dir, "gmx_infiles/md.mdp")),
                                                               walltime_per_part=0.005,
                                                               
                                                              )
               for _ in range(n_engines)
               ]

Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.
Neither max_frames nor max_steps given. Setting max_frames to infinity.


In [17]:
#initial_tp_mdt = mdt.load(os.path.join(scratch_dir, "gmx_infiles/ala_400K_TP_low_barrier.h5"))

In [18]:
#initial_tp_mdt[10].save_trr(os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"))

In [16]:
starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.trr"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/tp_frame.gro"))

starting_conf = aimmdd.Trajectory(trajectory_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"),
                                  structure_file=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

In [17]:
wrapped_alphaR

TrajectoryFunctionWrapper(function=<function alpha_R at 0x7fe1fe04e700>, call_kwargs={'scratch_dir': '/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'})

In [18]:
import mdtraj as mdt

In [19]:
mdt.load(os.path.join(scratch_dir, "gmx_infiles/conf.gro"), top=os.path.join(scratch_dir, "gmx_infiles/conf.gro"))

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (ALA, ACE)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (NME, ALA)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/formats/gro.py:307: UserWarning: WARNING: two consecutive residues with same number (HOH, NME)
  warnings.warn("WARNING: two consecutive residues with same number (%s, %s)" % (thisresname, residue.name))
/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/core/trajectory.py:422: UserWarning: top= kwarg igno

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7fe126c51d30>

In [20]:
mdt.load(starting_conf.trajectory_file, top=starting_conf.structure_file)

<mdtraj.Trajectory with 1 frames, 1651 atoms, 546 residues, and unitcells at 0x7fe126bff550>

In [21]:
alpha_R(starting_conf, **{"scratch_dir": scratch_dir})

/home/tb/hejung/.conda/envs/aimmd_nature_publish/lib/python3.8/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


array([ True])

In [22]:
await wrapped_alphaR(starting_conf)

array([ True])

In [23]:
scratch_dir

'/home/tb/hejung/DATA/aimmd_scratch/aimmd_distributed/'

In [24]:
tasks = [asyncio.create_task(p.propagate_and_concatenate(starting_configuration=starting_conf,
                                                         workdir=wdir,
                                                         deffnm="blib",
                                                         tra_out=os.path.join(wdir, "blub_until_state.trr"),
                                                          ))
         for p, wdir, mdp in zip(propagators, wdirs, mdps)]

done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)

In [25]:
list(done)[0]

<Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>

In [26]:
tasks

[<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>,
 <Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() 

In [27]:
done, pending = await asyncio.wait(tasks,)# return_when=asyncio.FIRST_COMPLETED)

In [28]:
for i in done:
    print(i)
    print(tasks.index(i))

<Task finished name='Task-5' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
2
<Task finished name='Task-6' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
3
<Task finished name='Task-3' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() got an unexpected keyword argument 'run_config'")>
0
<Task finished name='Task-4' coro=<PropagatorUntilAnyState.propagate_and_concatenate() done, defined at /home/tb/hejung/Documents/sources/aimmd/aimmd/distributed/logic.py:1311> exception=TypeError("propagate() g

In [29]:
pending

set()

In [21]:
start = time.time()
result1 = await wrapped_func(trajs[-1])
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.48328495025634766 s


In [22]:
start = time.time()
results = await asyncio.gather(*(wrapped_func(t) for t in trajs))
end = time.time()
print(f"time elapsed: {end-start} s")

time elapsed: 0.5739212036132812 s


In [23]:
trajs

[Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir0/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir1/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir2/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir3/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.part0002.trr, topology_file=/home/think/scratch/arcd_distributed/engine_wdir4/blub.tpr),
 Trajectory(trajectory_file=/home/think/scratch/arcd_distributed/engine_wdir5/blub.part0002.trr, topology_file=/home/think/scratch/arcd_dist